<a href="https://colab.research.google.com/github/cmp-math/PIC-Math-Workshop-on-Data-Science-2022/blob/main/KNNeighbors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import numpy as np
import pandas as pa
import sklearn
import matplotlib.pylab as py
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap

import pandas as pd
url = 'https://raw.githubusercontent.com/rcpaffenroth/PIC-Math-Workshop-on-Data-Science-2022/main/data/lithium_ion_data.csv'
data = pd.read_csv(url)
data['Crystal System (#)'] = data['Crystal System'].map({'monoclinic':0, 'orthorhombic':1, 'triclinic':2})
data.sample(10)

,Materials Id,Formula,Spacegroup,Formation Energy (eV),E Above Hull (eV),Band Gap (eV),Nsites,Density (gm/cc),Volume,Has Bandstructure,Crystal System,Crystal System (#)
263,mp-762860,LiCoSiO4,Pcmn,-2.339,0.097,0.736,28,3.605,291.060,True,orthorhombic,1
229,mp-868335,Li2Fe2Si3O10,F2dd,-2.647,0.092,3.040,34,2.286,537.355,True,orthorhombic,1
51,mp-762554,Li2FeSiO4,P21/c,-2.603,0.023,3.448,32,3.028,354.961,False,monoclinic,0
294,mp-773691,LiFe(SiO3)2,P1,-2.785,0.036,2.596,80,3.251,878.288,False,triclinic,2
70,mp-767938,LiFeSiO4,P21/c,-2.562,0.059,2.871,28,2.843,361.839,True,monoclinic,0
71,mp-762650,Li3Fe2(SiO4)2,P21,-2.564,0.060,1.331,30,3.026,347.513,True,monoclinic,0
226,mp-763624,Li2Fe(SiO3)2,Pm21n,-2.726,0.080,2.880,66,3.017,732.676,True,orthorhombic,1
260,mp-762888,LiCoSiO4,Cmcm,-2.351,0.084,0.871,14,3.762,139.432,True,orthorhombic,1
246,mp-763240,LiCoSiO4,C2221,-2.374,0.062,1.265,14,2.980,176.010,True,orthorhombic,1
219,mp-773468,LiFeSiO4,Imma,-2.546,0.076,1.311,14,3.898,131.954,True,orthorhombic,1


In [2]:
%matplotlib inline

In [3]:
# # Import the K-NN solver
from sklearn import neighbors
clf = neighbors.KNeighborsClassifier(n_neighbors=3)

In [30]:
# This is the one complicated bit of code in the whole demo.  There is no need to modify it, but please feel free if you want!
def runTest(clf,X,y,trainingPercent=0.66,plotTest=True):
    # A little cheat to make the pictures consistent
    np.random.seed(1234)
    
    # If there is one thing that I want to harp on, it is the difference
    # between testing and training errors!  So, here we create a training
    # set on which we computer the parameters of our algorithm, and a 
    # testing set for seeing how well we generalize (and work on real 
    # world problems).
    perm = np.random.permutation(len(y))
    n = X.shape[0]
    trainSize = int(trainingPercent*n)
    Xtrain = X[perm[:trainSize],0:2]
    Xtest = X[perm[trainSize:],0:2]

    yHat = y
    yHattrain = yHat[perm[:trainSize]]
    yHattest = yHat[perm[trainSize:]]

    # Run the calculation!
    clf.fit(Xtrain, yHattrain)
    
    # step size in the mesh for plotting the decision boundary.
    h = .02  
    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, m_max]x[y_min, y_max].
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                             np.arange(y_min, y_max, h))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)

    py.figure(figsize=(8, 6))
    cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
    cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])
    py.pcolormesh(xx, yy, Z, cmap=cmap_light, shading='auto')

    # Plot also the training points
    py.scatter(Xtrain[:, 0], Xtrain[:, 1], c=yHattrain, cmap=cmap_bold,marker='o')
    if plotTest:
        py.scatter(Xtest[:, 0], Xtest[:, 1], c=yHattest, cmap=cmap_bold,marker='+')
    py.xlim(xx.min(), xx.max())
    py.ylim(yy.min(), yy.max())
    # py.show()

    # Print out some metrics
    print('training score',clf.score(Xtrain,yHattrain))
    print('testing score',clf.score(Xtest,yHattest))

In [42]:
#X = data[['Formula', 'Spacegroup', 'Formation Energy (eV)', 'E Above Hull (eV)', 'Band Gap (eV)', 'Nsites', 'Density (gm/cc)', 'Volume', 'Has Bandstructure']]
#y = data['Crystal System']
X = np.array(data.iloc[:, :-1])
y = np.array(data.iloc[:, -1])
#model = method.fit(train[['Formula', 'Spacegroup', 'Formation Energy (eV)', 'E Above Hull (eV)', 'Band Gap (eV)', 'Nsites', 'Density (gm/cc)', 'Volume', 'Has Bandstructure']].to_numpy(),train['Crystal System'].to_numpy())

In [47]:
#yEasy = y.copy()
#yEasy[y==2] = 1
runTest(clf,X[:,:],y,plotTest=False)

#runTest(clf,X[:,[4,5]],y,plotTest=False)

ValueError: ignored